# Version 1

In [2]:
# ============================================
# Physics-aware dynamic batch EIS fitter (TL)
# Rs + (Rp || CPE) + TL   —   Nanoporous gold in H2SO4
# Option B rewrite: length-agnostic per-file allocations + pandas future-proofing
# ============================================

import numpy as np, pandas as pd, matplotlib.pyplot as plt, math, os, re, traceback
from scipy.optimize import least_squares
from pathlib import Path
from tqdm.auto import tqdm

# ------------------------- User knobs -------------------------
ROOT_DIR   = Path("/Users/hosseinostovar/Desktop/BACKUP/Data_H2SO4_NPG/data/NPG-500mM-H2SO4-whole_three")
FILE_GLOB  = "EIS_whole_spectrum_*_H2SO4_*C_*kHz_0.1Hz_pH=*.csv"

SHOW_PLOTS    = False
SAVE_FIG      = True
SAVE_CSV      = True
TARGET_RMSE   = 0.5
MAX_RETRIES   = 8
JITTER_SCALE  = 0.28
RNG_SEED      = 123
FREQ_UNIT_HINT= "auto"  # "auto","hz","khz","mhz"

# ----- Physics-aware controls -----
PRIOR_STRENGTH_MIN  = 0.01
PRIOR_STRENGTH_MAX  = 0.20
PRIOR_STRENGTH_STEP = 0.05
PRIOR_STRENGTH_MAX_HARD = 0.40   # absolute maximum clamp

NARROWING_FACTOR_DECADES = 0.6    # ± decades around prior center for log-params
NEIGHBOR_WINDOW = 3               # use up to last K concentration groups
TREND_PENALTY_LAMBDA = 0.10       # score = RMSE + λ * PhysicsRMS

# Minimum span floors after narrowing (transform space)
MIN_SPAN_LOG   = 0.40  # ~±0.20 ln (≈ ±22%)
MIN_SPAN_LOGIT = 0.35  # for n0/n1; Δn ≈ 0.05–0.08 near 0.7–0.9
MIN_NEIGHBORS  = 2     # need this many to start narrowing

# --------------------- Helper: hyper-parameter grid ---------------------
def build_hyper_grid():
    # prior_strength values: include 0.01, then 0.05..0.20 in 0.05 steps
    vals = [PRIOR_STRENGTH_MIN]
    s = PRIOR_STRENGTH_STEP
    x = 0.05
    while x <= PRIOR_STRENGTH_MAX + 1e-12:
        if x not in vals:
            vals.append(round(x, 2))
        x += s
    # Cartesian product over robust, weight_hf, prior_strength
    grid = []
    for robust in (True, False):
        for weight_hf in (True, False):
            for ps in vals:
                grid.append(dict(robust=robust, weight_hf=weight_hf, prior_strength=ps))
    return grid

# ------------------------- Flexible CSV reader -------------------------
def _find_column(cols, *cands):
    cl = [c.lower() for c in cols]
    for cand in cands:
        for i, c in enumerate(cl):
            if cand in c:
                return cols[i]
    return None

def _apply_freq_unit(freq, header_lower, unit_hint="auto"):
    if unit_hint and unit_hint.lower() in {"hz", "khz", "mhz"}:
        u = unit_hint.lower()
        if u == "khz": return freq * 1e3
        if u == "mhz": return freq * 1e6
        return freq
    if "khz" in header_lower: return freq * 1e3
    if "mhz" in header_lower: return freq * 1e6
    return freq

def load_impedance_csv(path, freq_unit_hint=FREQ_UNIT_HINT):
    df = pd.read_csv(path)
    cols = list(df.columns)

    fcol = _find_column(cols, "frequency", "freq", "hz", "khz", "mhz")
    if fcol is None:
        raise ValueError("No frequency column found")
    # pandas future-proof: pass dtype via keyword; avoid positional
    freq = df[fcol].to_numpy(dtype=float)
    freq = _apply_freq_unit(freq, fcol.lower(), unit_hint=freq_unit_hint)

    zrcol = None
    for key in ["we.z'", "z' (", "z real", "zreal"]:
        zrcol = _find_column(cols, key)
        if zrcol: break
    if zrcol is None: 
        raise ValueError("No Z' (real) column found")
    Zr = df[zrcol].to_numpy(dtype=float)

    zim_neg = _find_column(cols, "-z\"", "-z''", "we.-z\"")
    zim_pos = _find_column(cols, "z\"", "z''")
    if zim_neg is not None:
        Zi = -df[zim_neg].to_numpy(dtype=float)
    elif zim_pos is not None:
        Zi = -df[zim_pos].to_numpy(dtype=float)
    else:
        zim = _find_column(cols, "imag")
        if zim is None: 
            raise ValueError("No imaginary part column found")
        # Some exports store +Imag; model expects Nyquist with -Imag
        Zi = df[zim].to_numpy(dtype=float)
    Z = Zr + 1j*Zi
    return freq, Z

# -------------------------- Circuit elements --------------------------
def coth(x):
    t = np.tanh(x)
    out = np.empty_like(t, dtype=complex)
    small = np.abs(t) < 1e-12
    out[~small] = 1.0 / t[~small]
    xs = x[small]
    out[small] = 1.0/np.where(xs==0, 1e-30, xs) + xs/3.0
    return out

def zarc(Rp, Y0, n, w):
    return 1.0 / (1.0/Rp + Y0*(1j*w)**n) if Rp>0 else np.zeros_like(w, dtype=complex)

def tl_impedance(r, y0, n, L, w):
    gamma = np.sqrt(r*y0*(1j*w)**n)
    Z0 = np.sqrt(r/(y0*(1j*w)**n))
    return Z0 * coth(L*gamma)

# -------------------------- Transforms --------------------------
def _logit(x):      return np.log(x/(1.0-x))
def _invlogit(t):   return 1.0/(1.0+np.exp(-t))

# -------------------------- Physical safety bounds --------------------------
PHYS_BOUNDS_TL = {
    "Rs_min": 1e-3,  "Rs_max": 1e5,
    "Rp_min": 1e-2,  "Rp_max": 1e7,

    "Y0_min": 1e-8,  "Y0_max": 1e-2,
    "n0_min": 0.55,  "n0_max": 0.97,

    "r_min":  1e4,   "r_max": 1e8,    # Ω/m
    "y0_min": 1e-3,  "y0_max": 1e3,   # Ω^-1 s^n1 / m
    "n1_min": 0.60,  "n1_max": 0.97,

    "L_min":  1e-7,  "L_max": 3e-4,   # meters
}

def _bounds_TL():
    b = PHYS_BOUNDS_TL
    lb = np.array([
        np.log(b["Rs_min"]), np.log(b["Rp_min"]), np.log(b["Y0_min"]),
        _logit(b["n0_min"]),
        np.log(b["r_min"]),  np.log(b["y0_min"]),
        _logit(b["n1_min"]),
        np.log(b["L_min"])
    ], float)
    ub = np.array([
        np.log(b["Rs_max"]), np.log(b["Rp_max"]), np.log(b["Y0_max"]),
        _logit(b["n0_max"]),
        np.log(b["r_max"]),  np.log(b["y0_max"]),
        _logit(b["n1_max"]),
        np.log(b["L_max"])
    ], float)
    return lb, ub

# -------------------------- Model in transform space --------------------------
def model_TL(p, w):
    # p = [log_Rs, log_Rp, log_Y0, n0_logit, log_r, log_y0, n1_logit, log_L]
    Rs, Rp, Y0, n0 = np.exp(p[0]), np.exp(p[1]), np.exp(p[2]), _invlogit(p[3])
    r, y0, n1, L   = np.exp(p[4]), np.exp(p[5]), _invlogit(p[6]), np.exp(p[7])
    return Rs + zarc(Rp, Y0, n0, w) + tl_impedance(r, y0, n1, L, w)

# -------------------------- Initial guess --------------------------
def initial_guess(freq, Z):
    idx = np.argsort(freq)[::-1]; f = freq[idx]; Zs = Z[idx]
    hi_n = max(3, len(Zs)//20)
    Rs0 = np.percentile(Zs.real[:hi_n], 10)
    kmax = int(np.argmax(-Zs.imag)) if Zs.size else None
    fpk = f[kmax] if (kmax is not None and 0 <= kmax < f.size) else (np.median(f) if f.size else 1.0)
    n0, Rp0 = 0.8, max(np.percentile(Zs.real,90)-Rs0, 1.0)
    Y00 = 1.0/(Rp0*(2*np.pi*max(fpk,1e-9))**n0)
    r0 = max((np.max(Zs.real)-np.min(Zs.real))/50.0, 1.0)
    y0, n1 = 1e-2, 0.85
    wmin = 2*np.pi*max(min(f) if f.size else 1.0, 1e-3)
    L0 = 1.0/np.sqrt(max(r0*y0*(wmin**n1), 1e-12))
    p0 = np.array([
        np.log(max(Rs0,1e-3)), np.log(max(Rp0,1e-2)), np.log(max(Y00,1e-9)),
        math.log(n0/(1-n0)),
        np.log(max(r0,1e0)),   np.log(max(y0,1e-8)),
        math.log(n1/(1-n1)),
        np.log(max(L0,1e-8))
    ], float)
    # clip to physical bounds (coarse)
    ph_lb, ph_ub = _bounds_TL(); eps=1e-12
    return np.clip(p0, ph_lb+eps, ph_ub-eps)

# -------------------------- Physics expectations --------------------------
# name: (index in p, monotone vs C, monotone vs T, transform type)
EXPECT = {
    "Rs":        (0, -1, -1, "log"),
    "Rp":        (1, -1, -1, "log"),
    "Y0_ZARC":   (2, +1, +1, "log"),
    "n0":        (3,  0,  0, "logit"),
    "r_line":    (4, -1, -1, "log"),
    "y0_line":   (5, +1, +1, "log"),
    "n1":        (6,  0,  0, "logit"),
    "L":         (7,  0,  0, "log"),
}

# -------------------------- Dynamic priors --------------------------
def build_dynamic_priors(prev_df, C, T, p_names):
    """
    Build prior centers/sigmas from neighbors (<= C, last NEIGHBOR_WINDOW concentrations; weight by ΔC & ΔT).
    Returns dict: name -> {mu, sigma, type}.
    """
    priors = {}
    if prev_df is None or prev_df.empty:
        return priors

    cand = prev_df[prev_df["C"] <= C].copy()
    if cand.empty:
        return priors

    uniqC = sorted(cand["C"].unique())
    useC = uniqC[-min(NEIGHBOR_WINDOW, len(uniqC)):]
    cand = cand[cand["C"].isin(useC)].copy()
    if cand.empty:
        return priors

    cand["w"] = 1.0 / (1.0 + (C - cand["C"]).abs() + 0.3*(T - cand["T"]).abs())

    for name in p_names:
        if name not in cand.columns: 
            continue
        s = cand[[name, "w"]].dropna()
        if s.empty: 
            continue
        # weighted-median-ish via repetition
        rep = np.repeat(s[name].values, np.clip((s["w"]*5).round().astype(int),1,10))
        mu = float(np.median(rep)) if rep.size>0 else float(s[name].median())

        # log/logit sigma estimates
        if EXPECT[name][3] == "log":
            sigma_ln = NARROWING_FACTOR_DECADES * np.log(10.0)
            priors[name] = dict(mu=mu, sigma=sigma_ln, type="log")
        else:  # logit
            mu_clip = min(max(mu, 0.05), 0.95)
            band = 0.07  # ~Δn
            t0 = _logit(min(max(mu_clip, 1e-6), 1-1e-6))
            t1 = _logit(min(max(mu_clip+band, 1e-6), 1-1e-6))
            priors[name] = dict(mu=mu_clip, sigma=abs(t1-t0), type="logit")

    # Optional: tighten n1 a touch
    if priors.get("n1") and priors["n1"]["type"] == "logit":
        priors["n1"]["sigma"] = max(0.25, min(priors["n1"]["sigma"], 0.35))

    return priors

def dynamic_bounds_from_priors(base_bounds, priors, neighbor_count: int = 0):
    """
    Intersect physical bounds with a narrowed window around prior centers.
    Keep a minimum width so x0 is always feasible.
    Only narrow if neighbor_count >= MIN_NEIGHBORS.
    """
    lb, ub = base_bounds
    lb = lb.copy(); ub = ub.copy()

    if neighbor_count < MIN_NEIGHBORS or not priors:
        return lb, ub

    for name, (idx, _sC, _sT, typ) in EXPECT.items():
        if name not in priors:
            continue
        pr = priors[name]
        if typ == "log":
            mu_log = np.log(max(pr["mu"], 1e-300))
            span   = max(pr.get("sigma", 0.0), MIN_SPAN_LOG)
            cand_lb, cand_ub = mu_log - span, mu_log + span
        else:
            mu_t   = _logit(min(max(pr["mu"], 1e-9), 1-1e-9))
            span_t = max(pr.get("sigma", 0.0), MIN_SPAN_LOGIT)
            cand_lb, cand_ub = mu_t - span_t, mu_t + span_t

        lb[idx] = max(lb[idx], cand_lb)
        ub[idx] = min(ub[idx], cand_ub)

    eps = 1e-12
    for i in range(len(lb)):
        if not np.isfinite(lb[i]) or not np.isfinite(ub[i]):
            lb[i] = base_bounds[0][i]
            ub[i] = base_bounds[1][i]
        if ub[i] - lb[i] < eps:
            mid = 0.5*(ub[i]+lb[i])
            span = max(MIN_SPAN_LOG, MIN_SPAN_LOGIT)
            lb[i] = mid - 0.5*span
            ub[i] = mid + 0.5*span
    return lb, ub

def physics_prior_residual(p, C, T, priors, prior_strength):
    """
    Fixed-length physics residuals:
      - Anchors: one per parameter that has a prior (Gaussian in transform space)
      - Monotonic vs C: one per parameter with signC!=0 AND a prior; value is
        relu((wrong - tol)/tol) so it's zero if within tolerance.
      -> Length is constant for a given file+prior set.
    Returns (residual_vector, residual_rms)
    """
    res = []

    # 1) Anchors (always one per param that has a prior)
    for name, (idx, _signC, _signT, typ) in EXPECT.items():
        if name not in priors:
            continue
        if typ == "log":
            val_log = p[idx]
            mu_log  = np.log(max(priors[name]["mu"], 1e-300))
            s_ln    = max(priors[name]["sigma"], 1e-9)
            res.append( prior_strength * ((val_log - mu_log) / s_ln) )
        else:
            val_t = p[idx]
            mu_t  = _logit(min(max(priors[name]["mu"], 1e-9), 1-1e-9))
            s_t   = max(priors[name]["sigma"], 1e-6)
            res.append( prior_strength * ((val_t - mu_t) / s_t) )

    # 2) One-sided monotonic nudges vs C (always present, zero when satisfied)
    for name, (idx, signC, _signT, typ) in EXPECT.items():
        if signC == 0 or name not in priors:
            continue
        val = p[idx]
        if typ == "log":
            mu = np.log(max(priors[name]["mu"], 1e-300)); tol = 0.5   # ~half ln-decade
        else:
            mu = _logit(min(max(priors[name]["mu"], 1e-9), 1-1e-9)); tol = 0.25

        wrong = (val - mu) * np.sign(signC)  # >0 means moving the wrong way
        # relu but always include one residual per param
        mono = max(0.0, (wrong - tol) / max(tol, 1e-12))
        res.append( prior_strength * mono )

    r = np.array(res, dtype=float) if res else np.zeros(0, dtype=float)
    rms = float(np.sqrt(np.mean(r**2))) if r.size else 0.0
    return r, rms


# -------------------------- Residual (data + physics) --------------------------
def residual_total(p, w, Zexp, weights, C, T, priors, prior_strength):
    """
    Length-agnostic residual builder: all per-file allocations are derived from len(w).
    """
    Zm = model_TL(p, w)
    # data residuals (size 2*N)
    r_re = (Zm.real - Zexp.real)
    r_im = (Zm.imag - Zexp.imag)
    if weights is not None:
        # allocate per-file weight vector of size 2*N
        w2 = np.concatenate([weights, weights])
        r = np.concatenate([r_re, r_im]) * w2
    else:
        r = np.concatenate([r_re, r_im])

    # physics residuals (variable length)
    r_phys, _ = physics_prior_residual(p, C, T, priors, prior_strength)
    # single concat (length-agnostic)
    return np.concatenate([r, r_phys]) if r_phys.size else r

# -------------------------- Solve one spectrum --------------------------
def solve_one(freq, Z, C, T, rng, cfg, dyn_lb, dyn_ub,
              target_rmse=TARGET_RMSE, max_retries=MAX_RETRIES, jitter_scale=JITTER_SCALE):
    # sort ascending by f for evaluation; all per-file
    order = np.argsort(freq)
    freq = freq[order]; Z = Z[order]
    w = 2*np.pi*freq
    N = w.size

    # weights (length N) — allocate per file
    weights = None
    if cfg["weight_hf"]:
        mag = np.abs(Z)
        floor = np.percentile(mag, 5)
        weights = 1.0 / np.maximum(mag, floor)

    priors = cfg.get("_priors_", {})
    prior_strength = float(np.clip(cfg["prior_strength"], PRIOR_STRENGTH_MIN, PRIOR_STRENGTH_MAX_HARD))

    # initial guess clipped into (dynamic) bounds
    p_best = initial_guess(freq, Z)
    eps = 1e-12
    p0 = np.clip(p_best, dyn_lb+eps, dyn_ub-eps)

    def _solve(p0_local):
        res = least_squares(
            residual_total, p0_local,
            args=(w, Z, weights, C, T, priors, prior_strength),
            loss=("soft_l1" if cfg["robust"] else "linear"),
            max_nfev=20000, bounds=(dyn_lb, dyn_ub)
        )
        Zfit = model_TL(res.x, w)

        # compute data RMSE in a length-agnostic way
        d_re = (Zfit.real - Z.real)
        d_im = (Zfit.imag - Z.imag)
        if weights is not None:
            d = np.concatenate([d_re, d_im]) * np.concatenate([weights, weights])
        else:
            d = np.concatenate([d_re, d_im])
        rmse = float(np.sqrt(np.mean(d**2)))

        _, phys_rms = physics_prior_residual(res.x, C, T, priors, prior_strength)
        score = rmse + TREND_PENALTY_LAMBDA*phys_rms
        return dict(res=res, Zfit=Zfit, rmse=rmse, phys_rms=phys_rms, score=score)

    # First attempt (with safety fallback)
    try:
        best = _solve(p0)
    except ValueError as e:
        if "`x0` is infeasible" in str(e):
            p_mid = 0.5*(dyn_lb + dyn_ub)
            best = _solve(p_mid)
        else:
            raise

    # Retry loop with jitter (respect bounds)
    tries = 0
    exp_idx = np.array([0,1,2,4,5,7], dtype=int)
    logit_idx = np.array([3,6], dtype=int)
    while best["rmse"] > target_rmse and tries < max_retries:
        tries += 1
        mult = np.exp(rng.normal(0.0, jitter_scale, size=exp_idx.size))
        p_try = best["res"].x.copy()
        p_try[exp_idx] += np.log(mult)
        p_try[logit_idx] += rng.normal(0.0, jitter_scale, size=logit_idx.size)
        p_try = np.clip(p_try, dyn_lb+eps, dyn_ub-eps)
        cand = _solve(p_try)
        if cand["score"] < best["score"]:
            best = cand

    return best

# -------------------------- Ordering by C then T --------------------------
def parse_C_T_from_path(path: Path):
    s = str(path)
    C = None; T = None
    # Prefer folder names
    for part in path.parts:
        mC = re.search(r"(\d+(?:\.\d+)?)\s*mM", part, flags=re.IGNORECASE)
        if mC: C = float(mC.group(1))
        mT = re.search(r"(\d+(?:\.\d+)?)\s*C", part, flags=re.IGNORECASE)
        if mT: T = float(mT.group(1))
    # Fallback to filename
    if C is None:
        mC = re.search(r"(\d+(?:\.\d+)?)\s*mM", path.name, flags=re.IGNORECASE)
        if mC: C = float(mC.group(1))
    if T is None:
        mT = re.search(r"(\d+(?:\.\d+)?)\s*C", path.name, flags=re.IGNORECASE)
        if mT: T = float(mT.group(1))
    return C, T

def list_files_ordered(root: Path, glob: str):
    files = sorted(root.rglob(glob))
    rows = []
    for f in files:
        C, T = parse_C_T_from_path(f)
        if C is None or T is None:
            continue
        rows.append((C, T, f))
    rows.sort(key=lambda x: (x[0], x[1]))  # numeric sort: C then T
    return rows

# -------------------------- Batch runner --------------------------
def batch_fit_physics_aware(root_dir: Path = ROOT_DIR, file_glob: str = FILE_GLOB):
    results = []
    rng = np.random.default_rng(RNG_SEED)
    prev_df = pd.DataFrame(columns=["C","T","Rs","Rp","Y0_ZARC","n0","r_line","y0_line","n1","L"])

    files = list_files_ordered(root_dir, file_glob)
    if not files:
        print(f"[warn] no files matching {file_glob} under {root_dir}")
        return pd.DataFrame()

    base_lb, base_ub = _bounds_TL()
    HYPER_GRID = build_hyper_grid()

    for C, T, f in tqdm(files, desc="Physics-aware fitting", unit="file"):
        try:
            freq, Z = load_impedance_csv(str(f), freq_unit_hint=FREQ_UNIT_HINT)
            # fully per-file masks/allocations
            good = np.isfinite(freq) & np.isfinite(Z.real) & np.isfinite(Z.imag) & (freq>0)
            freq, Z = freq[good], Z[good]
            if freq.size < 5:
                raise ValueError("Too few valid points")

            # Build priors & dynamic bounds
            priors = build_dynamic_priors(prev_df, C, T, p_names=list(EXPECT.keys()))
            neighbor_count = 0 if prev_df is None or prev_df.empty else len(prev_df[prev_df["C"] <= C])
            dyn_lb, dyn_ub = dynamic_bounds_from_priors((base_lb, base_ub), priors, neighbor_count)

            # Order hyper-grid: for low C, try weight_hf=False combos earlier
            grid = list(HYPER_GRID)
            if C <= 5.0:
                grid.sort(key=lambda g: (g["weight_hf"], g["robust"], g["prior_strength"]))  # False first

            best_overall = None; best_cfg = None
            for cfg in grid:
                cfg = cfg.copy()
                cfg["_priors_"] = priors
                cfg["prior_strength"] = float(np.clip(cfg["prior_strength"], PRIOR_STRENGTH_MIN, PRIOR_STRENGTH_MAX_HARD))

                res = solve_one(freq, Z, C, T, rng, cfg, dyn_lb, dyn_ub,
                                target_rmse=TARGET_RMSE, max_retries=MAX_RETRIES, jitter_scale=JITTER_SCALE)
                if (best_overall is None) or (res["score"] < best_overall["score"]):
                    best_overall = res
                    best_cfg = cfg

            # Unpack best
            p = best_overall["res"].x
            Rs, Rp, Y0, n0 = float(np.exp(p[0])), float(np.exp(p[1])), float(np.exp(p[2])), float(_invlogit(p[3]))
            r, y0, n1, L   = float(np.exp(p[4])), float(np.exp(p[5])), float(_invlogit(p[6])), float(np.exp(p[7]))
            Zfit           = best_overall["Zfit"]
            rmse           = best_overall["rmse"]
            phys_rms       = best_overall["phys_rms"]

            # Bound hits?
            eps = 1e-9
            hit_bounds = any(abs(p[i]-dyn_lb[i])<eps or abs(p[i]-dyn_ub[i])<eps for i in range(len(p)))

            # Save outputs
            out_dir = f.parent / "fitting_TL" / f.stem
            out_dir.mkdir(parents=True, exist_ok=True)

            if SAVE_FIG or SHOW_PLOTS:
                fig = plt.figure(figsize=(6,5))
                plt.plot(Z.real, -Z.imag, 'o', ms=4, label="Data")
                plt.plot(Zfit.real, -Zfit.imag, '-', label="Fit")
                plt.xlabel("Z' (Ω)"); plt.ylabel("-Z'' (Ω)")
                plt.title(f"Nyquist: {f.name}")
                plt.legend(); plt.grid(True); plt.tight_layout()
                if SAVE_FIG:
                    fig.savefig(out_dir / "nyquist_TL.png", dpi=300)
                if SHOW_PLOTS: plt.show()
                else: plt.close(fig)

            meta = {
                "File": str(f),
                "C (mM)": C, "T (C)": T,
                "Model": "Rs+(Rp||CPE)+TL",
                "RMSE (Ω)": rmse, "PhysicsRMS": phys_rms,
                "ROBUST": best_cfg["robust"], "WEIGHT_HF": best_cfg["weight_hf"],
                "prior_strength": best_cfg["prior_strength"],
                "hit_bounds": hit_bounds,
                "Rs (Ω)": Rs, "Rp (Ω)": Rp,
                "Y0_ZARC (Ω^-1 s^n0)": Y0, "n0 (-)": n0,
                "r_line (Ω/m)": r, "y0_line (Ω^-1 s^n1 / m)": y0,
                "n1 (-)": n1, "L (m)": L
            }
            results.append(meta)

            if SAVE_CSV:
                # per-file allocations for saving
                order = np.argsort(freq)
                df_fit = pd.DataFrame({
                    "Frequency (Hz)": freq[order],
                    "Zreal_raw (Ω)" : Z.real[order],
                    "-Zimag_raw (Ω)": -Z.imag[order],
                    "Zreal_fit (Ω)" : Zfit.real[order],
                    "-Zimag_fit (Ω)": -Zfit.imag[order],
                })
                df_fit.to_csv(out_dir / "raw_vs_fit_TL.csv", index=False)
                pd.DataFrame([meta]).to_csv(out_dir / "params_TL.csv", index=False)

            # Update neighbors pool (guard concat-on-empty to avoid pandas future warnings)
            tmp = pd.DataFrame([{
                "C": C, "T": T,
                "Rs": Rs, "Rp": Rp, "Y0_ZARC": Y0, "n0": n0,
                "r_line": r, "y0_line": y0, "n1": n1, "L": L
            }])
            if prev_df.empty:
                prev_df = tmp
            else:
                prev_df = pd.concat([prev_df, tmp], ignore_index=True)

        except Exception as e:
            tqdm.write(f"[error] {f.name}: {e}")
            traceback.print_exc(limit=1)

    # Write summary (write directly to path; no intermediate string)
    df_sum = pd.DataFrame(results)
    if not df_sum.empty:
        df_sum.to_csv(root_dir / "summary_fitting_TL_physics.csv", index=False)
    return df_sum

# -------------------------- Run --------------------------
if __name__ == "__main__":
    summary = batch_fit_physics_aware(ROOT_DIR, FILE_GLOB)
    print("Done. Rows:", 0 if summary is None else len(summary))


Physics-aware fitting:   0%|          | 0/260 [00:00<?, ?file/s]

Done. Rows: 260
